In [1]:
import json
import re
import telebot
from telebot import types
from back_end import *
from forecast_script import *
import os
import time

Dict_data = {}
Dict_params = {}

bot = telebot.TeleBot('your_key')

@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    bot.send_message(message.from_user.id, "Здравствуйте, напишите тикер акции, которую хотите спрогнозировать (пример = AAPL)", reply_markup=markup)


@bot.message_handler(content_types=['text'])
def get_ticker(message):
    global Dict_data
    List_data = ['', '', '', ''] #ticker, horizon, model
    #bot.send_photo(message.from_user.id, 'dgEI7PDozTI.jpg')
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    ticker = message.text
    if(check_ticker(ticker) == 'Wrong ticker'):
        bot.send_message(message.from_user.id, f"Некорректный тикер {ticker}, введите новый", reply_markup=markup)
        return

    List_data[0] = ticker
    Dict_data.update({message.from_user.id : List_data})
    bot.send_message(message.from_user.id, 'Укажите горизонт предсказания в днях (пример 7)', reply_markup=markup)
    bot.register_next_step_handler(message, get_horizon)


def get_horizon(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    global Dict_data

    if not Dict_data.get(message.from_user.id):
        bot.send_message(message.from_user.id, 'Возникла неполадка перевыберите ваш тикер', reply_markup=markup)
        bot.register_next_step_handler(message, get_ticker)
        return

    horizon = Dict_data.get(message.from_user.id)[1]

    if(message.text == 'Да' and horizon != ''):
        markup = telebot.types.InlineKeyboardMarkup()
        markup.add(telebot.types.InlineKeyboardButton(text='Sarimax', callback_data=3))
        markup.add(telebot.types.InlineKeyboardButton(text='SSSmuzi', callback_data=4))
        markup.add(telebot.types.InlineKeyboardButton(text='Catboost', callback_data=5))
        bot.send_message(message.from_user.id, 'Выберите модель', reply_markup=markup)
        return
    elif(message.text == 'Нет' and horizon != ''):
        Dict_data[message.from_user.id][1] = ''
        bot.send_message(message.from_user.id, 'Перевыберите число', reply_markup=markup)
        bot.register_next_step_handler(message, get_horizon)
        return

    horizon = check_horizon(message.text)
    if(horizon == -1):
        bot.send_message(message.from_user.id, 'Пожалуйста напишите положительное число', reply_markup=markup)
    if(horizon == -3):
        bot.send_message(message.from_user.id, 'Пожалуйста напишите число', reply_markup=markup)
    if(horizon == -2):
        btn1 = types.KeyboardButton('Да')
        btn2 = types.KeyboardButton('Нет')
        markup.add(btn1, btn2)
        bot.send_message(message.from_user.id, 'Число слишком большое. Вы уверены?', reply_markup=markup)
        Dict_data[message.from_user.id][1] = int(message.text)
        bot.register_next_step_handler(message, get_horizon)
        return

    Dict_data[message.from_user.id][1] = horizon

    markup = telebot.types.InlineKeyboardMarkup()
    markup.add(telebot.types.InlineKeyboardButton(text='Sarimax', callback_data=3))
    markup.add(telebot.types.InlineKeyboardButton(text='EXPONENTIAL_SMOOTHING', callback_data=4))
    markup.add(telebot.types.InlineKeyboardButton(text='Catboost', callback_data=5))
    bot.send_message(message.from_user.id, 'Выберите модель', reply_markup=markup)


def get_dop_params(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    global Dict_data
    params = message.text
    #TODO убрать дублирование
    if params.lower() == 'run':
        bot.send_message(message.from_user.id, 'Взял в работу ваши данные, мне надо немного подумать')
        run_forecast([message.from_user.id] + Dict_data[message.from_user.id])
        count = 0
        while(not os.path.exists(f"{message.from_user.id}_forecast.png") and count <=5):
            count += 1
            time.sleep(1)
        if (count > 5):
            bot.send_message(message.from_user.id, 'Произошла ошибка, попробуйте снова')
        else:
            with open (f"{message.from_user.id}_forecast.png", 'rb') as png:
                bot.send_photo(message.from_user.id, png)
            os.remove(f"{message.from_user.id}_forecast.png")

    if Dict_data[message.from_user.id][2] == 'SARIMA':
        dict_params = parse_params_sarimax(params)
        filename = f"{message.from_user.id}_kwargs_model.json"
        with open(filename, "w", encoding="utf-8") as file:
          json.dump(dict_params, file)
        bot.send_message(message.from_user.id, 'Взял в работу ваши данные, мне надо немного подумать')
        run_forecast([message.from_user.id] + Dict_data[message.from_user.id])
        count = 0
        while(not os.path.exists(f"{message.from_user.id}_forecast.png") and count <=5):
            count += 1
            time.sleep(1)
        if (count > 5):
            bot.send_message(message.from_user.id, 'Произошла ошибка, попробуйте снова')
        else:
            with open (f"{message.from_user.id}_forecast.png", 'rb') as png:
                bot.send_photo(message.from_user.id, png)
            os.remove(f"{message.from_user.id}_forecast.png")
        
    if Dict_data[message.from_user.id][2] == 'EXPONENTIAL_SMOOTHING':
        filename = f"{message.from_user.id}_kwargs_model.json"
        with open(filename, "w", encoding="utf-8") as file:
          json.dump(dict_params, file)



@bot.callback_query_handler(func=lambda call: True)
def query_handler(call):
    global Dict_data
    global Dict_params
    if not Dict_data.get(call.message.chat.id):
        bot.answer_callback_query(callback_query_id=call.id, text='Возникла неполадка, выберите данные занового')
        return
    if   call.data == '3':
        Dict_data[call.message.chat.id][2] = 'SARIMA'
    elif call.data == '4':
        Dict_data[call.message.chat.id][2] = 'EXPONENTIAL_SMOOTHING'
    elif call.data == '5':
        Dict_data[call.message.chat.id][2] = 'CATBOOST'


    elif call.data == '11' or call.data == '12':
        if call.data == '12':
            list_params_smooth = ['add', '']
        else:
            list_params_smooth = ['mul', '']
        Dict_params.update({call.message.chat.id : list_params_smooth })
        markup = telebot.types.InlineKeyboardMarkup()
        markup.add(telebot.types.InlineKeyboardButton(text='Да', callback_data=13))
        markup.add(telebot.types.InlineKeyboardButton(text='Нет', callback_data=14))
        bot.send_message(call.message.chat.id, f"Ваш тренд затухающий?", reply_markup=markup)
        return
    elif call.data == '13' or call.data == '14':
        if not Dict_params[call.message.chat.id]:
              bot.send_message(call.message.chat.id, f"Произошла утечка данных, перевыберите ваши параметры")
        if call.data == '13':
            Dict_params[call.message.chat.id][1] = True
        else:
            Dict_params[call.message.chat.id][1] = False
        filename = f"{call.message.chat.id}_kwargs_model.json"
        with open(filename, "w", encoding="utf-8") as file:
          json.dump({'trend': Dict_params[call.message.chat.id][0], 'damped_trend' : Dict_params[call.message.chat.id][1], 'initialization_method': 'estimated'}, file)
        bot.send_message(call.message.chat.id, 'Взял в работу ваши данные, мне надо немного подумать')
        run_forecast([call.message.chat.id] + Dict_data[call.message.chat.id])
        count = 0
        while(not os.path.exists(f"{call.message.chat.id}_forecast.png") and count <=5):
            count += 1
            time.sleep(1)
        if (count > 5):
            bot.send_message(call.message.chat.id, 'Произошла ошибка, попробуйте снова')
        else:
            with open (f"{call.message.chat.id}_forecast.png", 'rb') as png:
                bot.send_photo(call.message.chat.id, png)
            os.remove(f"{call.message.chat.id}_forecast.png")
        return


    bot.send_message(call.message.chat.id, f"Ваши данные :{Dict_data[call.message.chat.id][0]} {Dict_data[call.message.chat.id][1]} {Dict_data[call.message.chat.id][2]}")
    bot.edit_message_reply_markup(call.message.chat.id, call.message.message_id)

    if call.data ==   '3':
        bot.send_message(call.message.chat.id, f"Выберите дополнительные параметры для модели SARIMA (p d q), впишите их в одну строку через пробел -- например p=1 q=2 d=3, \
                         неуказанным параметрам будет присвоено дефолтное значение, все единицы! Отправьте run если хотите оставить всё по дефолту")
        bot.register_next_step_handler(call.message, get_dop_params)

    elif call.data == '4':
        markup = telebot.types.InlineKeyboardMarkup()
        markup.add(telebot.types.InlineKeyboardButton(text='Аддитивный', callback_data=11))
        markup.add(telebot.types.InlineKeyboardButton(text='Мультипликативный', callback_data=12))
        bot.send_message(call.message.chat.id, f"Выберите параметры для модели EXPONENTIAL_SMOOTHING\nВыберите trend", reply_markup=markup)

    elif (call.data == '5'):
        bot.send_message(call.message.chat.id, 'Взял в работу ваши данные, мне надо немного подумать')
        run_forecast([call.message.chat.id] + Dict_data[call.message.chat.id])
        count = 0
        while(not os.path.exists(f"{call.message.chat.id}_forecast.png") and count <=5):
            count += 1
            time.sleep(1)
        if (count > 5):
            bot.send_message(call.message.chat.id, 'Произошла ошибка, попробуйте снова')
        else:
            with open (f"{call.message.chat.id}_forecast.png", 'rb') as png:
                bot.send_photo(call.message.chat.id, png)
            os.remove(f"{call.message.chat.id}_forecast.png")


bot.polling(none_stop=True, timeout=123) #обязательная для работы бота часть


[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2015-03-02  28.350000  28.799500  28.157499  28.750999  28.750999  50406000
2015-03-03  28.817499  29.042500  28.525000  28.939501  28.939501  50526000
2015-03-04  28.848499  29.081499  28.625999  28.916500  28.916500  37964000
2015-03-05  28.981001  29.160000  28.911501  29.071501  29.071501  35918000
2015-03-06  29.100000  29.139000  28.603001  28.645000  28.645000  37592000
...               ...        ...        ...        ...        ...       ...
2017-02-28  42.367500  42.441502  42.071999  42.246498  42.246498  27662000
2017-03-01  42.569000  42.900002  42.451000  42.837502  42.837502  36374000
2017-03-02  42.815498  42.824501  42.436001  42.492500  42.492500  25018000
2017-03-03  42.446999  42.541000  42.235500  42.453999  42.453999  20100000
2017-03-06  42.3429

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so

[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2015-03-02  28.350000  28.799500  28.157499  28.750999  28.750999  50406000
2015-03-03  28.817499  29.042500  28.525000  28.939501  28.939501  50526000
2015-03-04  28.848499  29.081499  28.625999  28.916500  28.916500  37964000
2015-03-05  28.981001  29.160000  28.911501  29.071501  29.071501  35918000
2015-03-06  29.100000  29.139000  28.603001  28.645000  28.645000  37592000
...               ...        ...        ...        ...        ...       ...
2017-02-28  42.367500  42.441502  42.071999  42.246498  42.246498  27662000
2017-03-01  42.569000  42.900002  42.451000  42.837502  42.837502  36374000
2017-03-02  42.815498  42.824501  42.436001  42.492500  42.492500  25018000
2017-03-03  42.446999  42.541000  42.235500  42.453999  42.453999  20100000
2017-03-06  42.3429

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning:

overflow encountered in matmul

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning:

overflow encounte

[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2015-03-02  28.350000  28.799500  28.157499  28.750999  28.750999  50406000
2015-03-03  28.817499  29.042500  28.525000  28.939501  28.939501  50526000
2015-03-04  28.848499  29.081499  28.625999  28.916500  28.916500  37964000
2015-03-05  28.981001  29.160000  28.911501  29.071501  29.071501  35918000
2015-03-06  29.100000  29.139000  28.603001  28.645000  28.645000  37592000
...               ...        ...        ...        ...        ...       ...
2017-02-28  42.367500  42.441502  42.071999  42.246498  42.246498  27662000
2017-03-01  42.569000  42.900002  42.451000  42.837502  42.837502  36374000
2017-03-02  42.815498  42.824501  42.436001  42.492500  42.492500  25018000
2017-03-03  42.446999  42.541000  42.235500  42.453999  42.453999  20100000
2017-03-06  42.3429

...

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: V

[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2015-03-02  28.350000  28.799500  28.157499  28.750999  28.750999  50406000
2015-03-03  28.817499  29.042500  28.525000  28.939501  28.939501  50526000
2015-03-04  28.848499  29.081499  28.625999  28.916500  28.916500  37964000
2015-03-05  28.981001  29.160000  28.911501  29.071501  29.071501  35918000
2015-03-06  29.100000  29.139000  28.603001  28.645000  28.645000  37592000
...               ...        ...        ...        ...        ...       ...
2017-02-28  42.367500  42.441502  42.071999  42.246498  42.246498  27662000
2017-03-01  42.569000  42.900002  42.451000  42.837502  42.837502  36374000
2017-03-02  42.815498  42.824501  42.436001  42.492500  42.492500  25018000
2017-03-03  42.446999  42.541000  42.235500  42.453999  42.453999  20100000
2017-03-06  42.3429

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: Val

[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2015-03-02  32.312500  32.570000  32.075001  32.272499  29.002459  192386800
2015-03-03  32.240002  32.380001  32.022499  32.340000  29.063124  151265200
2015-03-04  32.275002  32.389999  32.080002  32.134998  28.878889  126665200
2015-03-05  32.145000  32.187500  31.440001  31.602501  28.400351  226068400
2015-03-06  32.099998  32.342499  31.565001  31.650000  28.443033  291368400
...               ...        ...        ...        ...        ...        ...
2017-02-28  34.270000  34.360001  34.174999  34.247501  32.010578   93931600
2017-03-01  34.472500  35.037498  34.400002  34.947498  32.664848  145658400
2017-03-02  35.000000  35.070000  34.689999  34.740002  32.470901  104844000
2017-03-03  34.695000  34.957500  34.647499  34.945000  32.662514   84432400
2017-03

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\kir12\.conda\envs\vega_time_series\lib